In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

D:\Anaconda\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [13]:
# 读数据
mnist = input_data.read_data_sets("C:\\Users\\Administrator\\Desktop\\MNIST_data",one_hot=True)
# 设置批次大小
batch_size =100
# 批次数量
n_batch = mnist.train.num_examples // batch_size

def Weights_varible(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)
def biases_varible(shape):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)#平均值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev',stddev)#标准差
        tf.summary.scalar('max',tf.reduce_max(var))#最大值
        tf.summary.scalar('min',tf.reduce_min(var))#最小值
        tf.summary.histogram('histogram',var)#直方图
# 定义命名空间
with tf.name_scope('input'):
    #定义占位符
    x = tf.placeholder(tf.float32,[None,784])
    y = tf.placeholder(tf.float32,[None,10])
# 定义卷积层
def conv2d(x,W):
    #输入
    #过滤器
    #[1,x,y,1]步长
    return tf.nn.conv2d(input=x,filter=W,strides=[1,1,1,1],padding="SAME")

def max_pool_2x2(x):
    #输入
    #窗口大小[1,x,y,1]
    #步长[1,x,y,1]
     return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
with tf.name_scope('input_reshape'):
    #改造数据
    x_images = tf.reshape(x,[-1,28,28,1])
#构建网络
#第一层
with tf.name_scope('layer'):
    with tf.name_scope('layer_conv1'):
        W_conv1 = Weights_varible([5,5,1,32])# 5*5窗口大小，输入平面，输出平面
        b_conv1 = biases_varible([32])
        h_conv1 = tf.nn.relu(conv2d(x_images,W_conv1)+b_conv1)
        h_pool1 = max_pool_2x2(h_conv1)

    with tf.name_scope('layer_conv2'):
        #第二层卷积
        W_conv2 = Weights_varible([5,5,32,64])# 5*5窗口大小，输入平面，输出平面
        b_conv2 = biases_varible([64])
        h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)
        h_pool2 = max_pool_2x2(h_conv2)

# # 第三层卷积
# W_conv3 = Weights_varible([5,5,64,128])# 5*5窗口大小，输入平面，输出平面
# b_conv3 = biases_varible([128])
# h_conv3 = tf.nn.relu(conv2d(h_pool2,W_conv3)+b_conv3)
# h_pool3 = max_pool_2x2(h_conv3)


#28*28图片第一次卷积后还是28*28，第一次池化后变为14*14
#第二次卷积后为14*14，第二次池化后变为了7*7
#经过上面操作后得到64张7*7的平面；
    with tf.name_scope('layer_fc1'):
        #初始化第一个全连接层
        W_fc1 = Weights_varible([7*7*64,512])
        b_fc1 = biases_varible([512])
        #数据扁平化
        h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)

        #保持输入神经元数量
        keep_prob= tf.placeholder(tf.float32)
        h_fc1_drop=tf.nn.dropout(h_fc1,keep_prob=keep_prob)
    with tf.name_scope('layer_fc2'):
        #初始化第二个全连接层
        W_fc2 = Weights_varible([512,10])
        b_fc2 = biases_varible([10])
        variable_summaries(W_fc2)
        variable_summaries(b_fc2)

        #得到预测值
        prediction = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2)+b_fc2)
with tf.name_scope('loss'):
    #损失函数
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
    tf.summary.scalar('loss',loss)
with tf.name_scope('train'):
    #训练
    train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
with tf.name_scope('accuracy'):
    # 计算准确率
    #最大值坐标
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
        
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar('accuracy',accuracy)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('logs/',sess.graph)
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.7})
        writer.add_summary(summary,epoch) 
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.})
        print("Iter  :"+str(epoch)+",Testing Accuracy:"+str(accuracy))
        

Extracting C:\Users\Administrator\Desktop\MNIST_data\train-images-idx3-ubyte.gz
Extracting C:\Users\Administrator\Desktop\MNIST_data\train-labels-idx1-ubyte.gz
Extracting C:\Users\Administrator\Desktop\MNIST_data\t10k-images-idx3-ubyte.gz
Extracting C:\Users\Administrator\Desktop\MNIST_data\t10k-labels-idx1-ubyte.gz


KeyboardInterrupt: 